Author:  Hao Jiang
## Goal for this notebook
Based on version1 data cleaning. This notebook tries to;
 * Further explore the data
 * Try some feature engineering

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import sklearn

from scipy.stats import skew
from scipy.stats.stats import pearsonr

%config InlineBackend.figure_format = 'retina' #set 'png' here when working on notebook
%matplotlib inline

In [2]:
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")
train.head()
# test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
# combine the data and take all of the feature. This is for easier feature engineering. 
all_data = pd.concat((train.iloc[:,1:-1], test.iloc[:,1:-1]))
Y = train["SalePrice"]

# remember the length of train data
trainLen = len(train)

Due to previous prediction, we could see that living area and other area infomation is a crucial predictor to final sold price. Insteed of the sold price, price per SF could be a better feature to compare the effect of other features. But first, we need to have some conversions on these continuouse data.  
To begin with, we can see that GrLivArea = 1stFlrSF + 2ndFlrSF, TotalBsmtSF = BsmtFinSF1 + BsmtFinSF2 + BsmtUnfSF. So if seems better to reduce the features here.

In [4]:
# There is a na TotalBsmtSF value in the test set for 
all_data.loc[all_data["TotalBsmtSF"].isnull(),'TotalBsmtSF'] = 0

all_data["1stFlrSFRatio"] = all_data["1stFlrSF"] * 1.0 / all_data["GrLivArea"]
all_data["BsmtUnfSFRatio"] = all_data["BsmtUnfSF"] * 1.0 / all_data["TotalBsmtSF"]
all_data["BsmtFinSF1Ratio"] = all_data["BsmtFinSF1"] * 1.0 / all_data["TotalBsmtSF"]
all_data["BsmtFinSF2Ratio"] = all_data["BsmtFinSF2"] * 1.0 / all_data["TotalBsmtSF"]
# BsmtFinSF1 & 2 would be latter merged with type

In [5]:
# Add a variable for total porch Area
all_data['TotalPorchSF'] = all_data[['OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch']].sum(axis = 1)
# Add a variable for total living area
all_data['TotalLivArea'] = all_data['GrLivArea'] + all_data['TotalBsmtSF']


In [6]:
# Drop already processed features
all_data = all_data.drop([
        '1stFlrSF','2ndFlrSF',
        'BsmtFinSF1','BsmtFinSF2','BsmtUnfSF',
        'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch'
], axis=1)

In [7]:
continuous_feats = pd.Index([name for name in all_data.columns if "Area" in name or "SF" in name] + ['LotFrontage', 'MiscVal'])
area_feats = pd.Index([name for name in continuous_feats if 'Ratio' not in name])
len(area_feats), len(continuous_feats)

(12, 16)

In [8]:
# matplotlib.rcParams['figure.figsize'] = (12.0, 18.0)

# # Non zeor non null histogram
# for i in range(len(area_feats)):
#     plt.subplot(4,3, i + 1)
#     plt.hist(all_data[area_feats[i]].loc[all_data[area_feats[i]] > 0], bins=20)
#     plt.title(area_feats[i])

Many of these values are skewed. Let's convert them (apart from the ratio features).

In [9]:
skewed_feats = all_data[area_feats].apply(lambda x: skew(x[x > 0]))
skewed_feats = skewed_feats[skewed_feats > 0.9].index
skewed_feats

Index([u'GrLivArea', u'LotArea', u'LowQualFinSF', u'MasVnrArea',
       u'TotalBsmtSF', u'WoodDeckSF', u'TotalPorchSF', u'TotalLivArea',
       u'LotFrontage', u'MiscVal'],
      dtype='object')

In [10]:
# Seems fine, let's convert. But before converting them that's make a feature for price per SF LivArea

YPerSF = Y / all_data['TotalLivArea'][:trainLen]
Y = np.log1p(Y)
YPerSF = np.log1p(YPerSF)
all_data[skewed_feats] = np.log1p(all_data[skewed_feats])

In [11]:
# # Not let's check the scatter of these values to price
# matplotlib.rcParams['figure.figsize'] = (12.0, 18.0)

# # Non zeor non null histogram
# for i in range(len(continuous_feats)):
#     plt.subplot(4,4, i + 1)
#     plt.scatter(y = Y, x = all_data[continuous_feats[i]][:trainLen])
#     plt.title(continuous_feats[i])

We can see that TotalLivingArea is Stronging correlated to price.

According to hint, there're some outliers. Let's check the distribution of total living area for both train and test set.

In [12]:
# matplotlib.rcParams['figure.figsize'] = (12.0, 6.0)
# plt.subplot(121)
# all_data[:trainLen]['TotalLivArea'].hist(bins = 20)
# plt.title('Train set total living area hist')
# plt.subplot(122)
# all_data[trainLen:]['TotalLivArea'].hist(bins = 20)
# plt.title('Test set total living area hist')

It seems that we could ignore data for log1p(TtllivingArea) < 6

In [13]:
all_data.index[all_data['TotalLivArea'] <= 6]

Int64Index([533], dtype='int64')

In [14]:
# Latter
# all_data[:trainLen] = all_data[:trainLen].drop([533], axis = 0).shape
# all_data[:trainLen].shape

Basically that's all for continuous data. Now that's convert Ordinal ones.

In [15]:
# LotShape
all_data['LotShape'] = all_data['LotShape'].replace(['IR3','IR2','IR1' ,'Reg'], [1,2,3,4])
all_data['LandContour'] = all_data['LandContour'].replace(['Low', 'HLS', 'Bnk','Lvl'], [1,2,3,4])
all_data['Utilities'] = all_data['Utilities'].replace(['ELO','NoSeWa','NoSewr','AllPub'], [1,2,3,4])
all_data['LandSlope'] = all_data['LandSlope'].replace(['Sev' , 'Mod', 'Gtl'], [1,2,3])
all_data['ExterQual'] = all_data['ExterQual'].replace(['Po', 'Fa', 'TA', 'Gd', 'Ex'], [1,2,3,4,5])
all_data['ExterCond'] = all_data['ExterCond'].replace(['Po', 'Fa', 'TA', 'Gd', 'Ex'], [1,2,3,4,5])
all_data['BsmtQual'] = all_data['BsmtQual'].replace(['Po', 'Fa', 'TA', 'Gd', 'Ex'], [1,2,3,4,5])
all_data['BsmtCond'] = all_data['BsmtCond'].replace(['Po', 'Fa', 'TA', 'Gd', 'Ex'], [1,2,3,4,5])
all_data['BsmtExposure'] = all_data['BsmtExposure'].replace(['No', 'Mn', 'Av', 'Gd'], [1,2,3,4])
all_data['BsmtFinType1'] = all_data['BsmtFinType1'].replace(['Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'], [1,2,3,4,5,6])
all_data['BsmtFinType2'] = all_data['BsmtFinType2'].replace(['Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'], [1,2,3,4,5,6])
all_data['HeatingQC'] = all_data['HeatingQC'].replace(['Po', 'Fa', 'TA', 'Gd', 'Ex'], [1,2,3,4,5])
all_data['KitchenQual'] = all_data['KitchenQual'].replace(['Po', 'Fa', 'TA', 'Gd', 'Ex'], [1,2,3,4,5])
all_data['Functional'] = all_data['Functional'].replace(['Sal','Sev','Maj2','Maj1','Mod','Min2','Min1','Typ'], [
        1,2,3,4,5,6,7,8])
all_data['FireplaceQu'] = all_data['FireplaceQu'].replace(['Po', 'Fa', 'TA', 'Gd', 'Ex'], [1,2,3,4,5])
all_data['GarageFinish'] = all_data['GarageFinish'].replace(['Unf', 'RFn', 'Fin'], [1,2,3])
all_data['GarageQual'] = all_data['GarageQual'].replace(['Po', 'Fa', 'TA', 'Gd', 'Ex'], [1,2,3,4,5])
all_data['GarageCond'] = all_data['GarageCond'].replace(['Po', 'Fa', 'TA', 'Gd', 'Ex'], [1,2,3,4,5])
all_data['PoolQC'] = all_data['PoolQC'].replace(['Fa', 'TA', 'Gd', 'Ex'], [1,2,3,4])
all_data['Fence'] = all_data['Fence'].replace(['MnWw', 'GdWo', 'MnPrv', 'GdPrv'], [1,2,3,4])

In [16]:
# Ordinal check
ord_feats = pd.Index(['LotShape','LandContour','Utilities','LandSlope',
                      'OverallQual','OverallCond','ExterQual','ExterCond',
                      'BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1',
                      'BsmtFinType2','HeatingQC','KitchenQual','Functional',
                      'FireplaceQu','GarageFinish','GarageQual','GarageCond',
                      'PoolQC','Fence'])
all_data[ord_feats] = all_data[ord_feats].fillna(0)
len(ord_feats)

22

In [17]:
# # Not let's check the boxplot of these values to price per sf
# matplotlib.rcParams['figure.figsize'] = (12.0, 24.0)

# # Non zeor non null histogram
# for i in range(len(ord_feats)):
#     plt.subplot(6,4, i + 1)
#     sns.boxplot(y = YPerSF, x = all_data[:trainLen][ord_feats[i]])

Generally, these ordinal data are correlated to price per SF, but not all of them are strictly as so.
Further more, sometimes na data does not seems to be worse.

We'll jsut try to see how these works first.

Now let's try to handle the discrete data.

In [18]:
# Let's add time interval in year between build and sold time.
all_data['BltYears'] = all_data['YrSold'] -  all_data['YearBuilt']
all_data['RemYears'] = all_data['YrSold'] -  all_data['YearRemodAdd']
all_data['GaBltyears'] = all_data['YrSold'] -  all_data['GarageYrBlt']
all_data = all_data.drop(['YearBuilt','YearRemodAdd','GarageYrBlt'], axis = 1)

In [19]:
# MoSold seems more like a categorical data, let's check the distribution
# sns.boxplot(y = Y, x = all_data[:trainLen]['MoSold'])
all_data['MoSold'] = all_data['MoSold'].astype('category')

In [20]:
all_data['CentralAir'] = all_data['CentralAir'].replace(['N','Y'], [0,1])

Finally let's fill in na data and one-hot encoding nomincal ones

In [21]:
# check na value count
nullCnt = pd.DataFrame({'nullNums' : all_data.isnull().sum()})
nullCnt['DataType'] = all_data[nullCnt.index].dtypes
print nullCnt[nullCnt['nullNums'] > 0].sort_values(by = 'DataType')

                 nullNums DataType
LotFrontage           486  float64
BsmtFinSF1Ratio        79  float64
BsmtUnfSFRatio         79  float64
MasVnrArea             23  float64
BsmtFinSF2Ratio        79  float64
GarageCars              1  float64
GaBltyears            159  float64
BsmtHalfBath            2  float64
BsmtFullBath            2  float64
GarageArea              1  float64
Exterior2nd             1   object
GarageType            157   object
Exterior1st             1   object
MSZoning                4   object
Electrical              1   object
MasVnrType             24   object
MiscFeature          2814   object
SaleCondition        1459   object
SaleType                1   object
Alley                2721   object


Based on previous data cleaning we know that we should fill in LatFrontage data.  
For GaBltyears, we will just set them to 999

In [22]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(all_data.loc[all_data['LotFrontage'].notnull(), ['LotArea']], all_data.loc[all_data['LotFrontage'].notnull(), ['LotFrontage']])
all_data.loc[all_data['LotFrontage'].isnull(), ['LotFrontage']] = reg.predict(all_data.loc[all_data['LotFrontage'].isnull(), ['LotArea']])

In [23]:
all_data.loc[all_data['GaBltyears'].isnull(), 'GaBltyears'] = 999
# Fill na with 0
all_data[all_data.columns[all_data.dtypes == 'float64']] = all_data[all_data.columns[all_data.dtypes == 'float64']].fillna(0)

In [24]:
# There're several pairs of feature that should be merged when one-hot encoded

# Deal with Exterior
for name in all_data["Exterior1st"].dropna().unique():
    all_data[name] = 1 * ((all_data["Exterior1st"] == name) | (all_data["Exterior2nd"] == name))

# Deal with Condition
for name in all_data["Condition1"].dropna().unique():
    all_data[name] = 1 * ((all_data["Condition1"] == name) | (all_data["Condition2"] == name))

In [25]:
# Drop those features that we have converted already

all_data = all_data.drop(['Exterior1st','Exterior2nd'], axis=1)
all_data = all_data.drop(['Condition1','Condition2'], axis=1)

In [26]:
# one-hot encode category data
all_data = pd.get_dummies(all_data)
all_data.shape
# all_data.head()

(2919, 204)